In [3]:
import numpy as np
from scipy.linalg import logm, expm
import random
import matplotlib.pyplot as plt

class LIM:
    """
    An example of linear inverse model (Markov Chain)

    Parameters
    ----------
    data: np.ndarray
        An array containing data (has dimension of [time X state] )
        The lead-lag linear regression will be based on this array.
    lag: int
        The lag of lead-lag linear regression
        lag = 0 will return G and G1 matrix with all elements are 1.
    """
    def __init__(self, data: np.ndarray, lag: int):
        self.data = data
        self.lag  = lag
        self.G    = None
        self.G1   = None

    def _calc_G1(self, G: np.ndarray, lag: int):
        """
        Calculating the lag 1 regression coefficient matrix (G1) by lag tau

        Parameters
        ----------
        None

        Returns
        -------
        None
        """
        return expm(logm(G)/lag)

    def _calc_G(self):
        """
        Calculating the regression coefficient matrix (G) of lead time and lag time

        Parameters
        ----------
        None

        Returns
        -------
        None
        """
        if self.lag != 0:
            x0 = np.copy(self.data[:, :-self.lag])
            xt = np.copy(self.data[:, self.lag:])
            c0 = np.matmul(x0, x0.T)
            ct = np.matmul(xt, x0.T)
            G  = np.matmul(ct, np.linalg.inv(c0))
            self.G  = G
            self.G1 = self._calc_G1(self.G, self.lag)
        else:
            self.G  = np.ones((self.data.shape[0], self.data.shape[0]))
            self.G1 = self.G

    def build(self):
        """
        Building the model
        Call _calc_G()
        Parameters
        ----------
        None

        Returns
        -------
        None
        """
        self._calc_G()


In [4]:
# using MJO index as an example
# download data from GitHub
!wget https://raw.githubusercontent.com/kuiper2000/LIM/main/OMI_index.txt
rmm2=[float(l.split()[4]) for l in open('/content/OMI_index.txt')]
rmm2=np.array(rmm2).reshape([1,13514])
rmm2=-rmm2
rmm1=[float(l.split()[5]) for l in open('/content/OMI_index.txt')]
rmm1=np.array(rmm1).reshape([1,13514])
RMM = np.concatenate((rmm1,rmm2),axis=0)

--2023-12-04 12:48:38--  https://raw.githubusercontent.com/kuiper2000/LIM/main/OMI_index.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 675699 (660K) [text/plain]
Saving to: ‘OMI_index.txt.1’

OMI_index.txt.1     100%[===================>] 659.86K  --.-KB/s    in 0.06s   

2023-12-04 12:48:38 (10.3 MB/s) - ‘OMI_index.txt.1’ saved [675699/675699]



In [33]:
model = LIM(RMM, lag = 4)
model.build()
G     = model.G
G1    = model.G1